In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery

Matplotlib is building the font cache; this may take a moment.


In [5]:
# Initialize BigQuery client
client = bigquery.Client(project='big-sales-data-453023')

# Fetch entire dataset from BigQuery
query = """
    SELECT * FROM `big-sales-data-453023.sales_data.sales`;
"""
data = client.query(query).to_dataframe()

/Users/siddharthgoradia/Documents/Loblaw-Data-Science/.venv/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
# Convert date columns to datetime format
data['order_date'] = pd.to_datetime(data['order_date'], errors='coerce')

# Extract required time-based features
data['month'] = data['order_date'].dt.month
data['hour'] = data['order_date'].dt.hour

# Compute total sales
data['total_sales'] = data['quantity_ordered'] * data['price_each']

# Monthly Sales Trend
monthly_sales = data.groupby('month')['total_sales'].sum().reset_index()
plt.figure(figsize=(10,5))
sns.lineplot(x=monthly_sales['month'], y=monthly_sales['total_sales'], marker='o')
plt.xlabel("Month")
plt.ylabel("Total Sales ($)")
plt.title("Monthly Sales Trend")
plt.xticks(range(1,13))
plt.grid()
plt.show()

# Peak Sales Hours
hourly_sales = data.groupby('hour')['total_sales'].sum().reset_index()
plt.figure(figsize=(10,5))
sns.barplot(x=hourly_sales['hour'], y=hourly_sales['total_sales'])
plt.xlabel("Hour of Day")
plt.ylabel("Total Sales ($)")
plt.title("Peak Sales Hours")
plt.xticks(range(0,24))
plt.grid()
plt.show()

# Top Selling Products
top_products = data.groupby('product_name').agg({
    'quantity_ordered': 'sum',
    'total_sales': 'sum'
}).reset_index().sort_values(by='total_sales', ascending=False).head(10)
plt.figure(figsize=(12,6))
sns.barplot(y=top_products['product_name'], x=top_products['total_sales'])
plt.xlabel("Total Revenue ($)")
plt.ylabel("Product Name")
plt.title("Top Selling Products")
plt.grid()
plt.show()

# Insights Summary
print("\nActionable Insights:")
print("1. December has the highest sales. Focus on holiday promotions.")
print("2. Peak sales occur between 6-9 PM. Optimize marketing campaigns for these hours.")
print("3. Top 5 products contribute ~50% of revenue. Prioritize inventory and promotions.")
